## Classification
#### Python notebook for Engel outcome prediction (same workflow for Naming outcome)
#### Scikit-Learn library

## **Import modules**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold, RepeatedStratifiedKFold, train_test_split,cross_val_predict
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, precision_score, recall_score, confusion_matrix

from sklearn.feature_selection import SelectFromModel

from numpy import savetxt, loadtxt

from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence

## **Import Data and prepocessing**

First, let's load the dataset.

In [ ]:
# Read data from csv
input_file = "./Data_Neuropsy.csv"
# semi-comma delimited is the default
data_raw = pd.read_csv(input_file, header = 0, delimiter=";")

#### **Drop samples with missing values**

In [ ]:
data_complet = data_raw.iloc[:,1:54].dropna()
X = data_complet.iloc[:,0:51]
# target variable
y = np.ravel(data_complet[["ENG_bin"]])

#### **Standard scaling for continuous variables**

In [ ]:
scaler = StandardScaler()

In [ ]:
X[['AGE','EDH','AHS','ASO','DUR','FRQ','AED','SEV','EDU']]=scaler.fit_transform(X[['AGE','EDH','AHS','ASO','DUR','FRQ','AED','SEV','EDU']])

In [ ]:
X.iloc[:,16:51]=scaler.fit_transform(X.iloc[:,16:51])

#### **Sub dataset selection :  PRI, VMI, VCI, AMI, NAM, SFL, PFL, TMT, STR**

In [ ]:
X_dataset = X[["PRI", "VMI", "VCI", "AMI", "NAM", "SFL", "PFL", "TMT", "STR"]]

In [ ]:
columns = X_dataset.columns

In [ ]:
X = X_dataset

## **Model Evaluation Using Cross-Validation**

In [ ]:
##########################################################################################
# model

model = LogisticRegression(penalty="l2", solver='liblinear', class_weight='balanced')

# evaluate the model using 10-fold cross-validation repeated 100 times and balanced accuracy

n_splits = 10
n_repeats = 100

scores=[]

for i in range (n_repeats):
    cv=StratifiedKFold(n_splits=n_splits, random_state=i, shuffle=True)
    scores_temp=[]
    scores_temp=cross_val_score(model, X, y, scoring='balanced_accuracy',cv=cv)
    scores.append(scores_temp.mean())      

scores = np.asarray(scores)

print (f'Overall Balanced Accuracy mean :{scores.mean():.2f}')
print (f'Overall Balanced Accuracy std :{scores.std():.2f}')

# evaluate the model using k-fold cross-validation repeated 1000 times and roc AUC

scores=[]

for i in range (n_repeats):
    cv=StratifiedKFold(n_splits=n_splits, random_state=i, shuffle=True)
    scores_temp=[]
    scores_temp=cross_val_score(model, X, y, scoring='roc_auc',cv=cv)
    scores.append(scores_temp.mean())      

scores = np.asarray(scores) 


print (f'Overall AUC mean :{scores.mean():.2f}')
print (f'Overall AUC std :{scores.std():.2f}')

print('\n')

## **Feature Selection workflow**

In [ ]:
X = X.to_numpy()

In [ ]:
%%time

# Algorithm choice
logisticL1 = LogisticRegression(penalty="l1", solver='liblinear', class_weight='balanced', n_jobs = -1)
logisticL2 = LogisticRegression(penalty="l2", solver='liblinear', class_weight='balanced', n_jobs = -1)
Tree = ExtraTreesClassifier(n_estimators=50, n_jobs = -1)

performance=[]
precision=[]
recall=[]
matrix = []

iterations = 100
features=[]


for i in range (1,iterations+1):
    # Cross-validation scheme
    cv=StratifiedKFold(n_splits=5, random_state=i, shuffle=True)
    
    for train_index, test_index in cv.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]  

        # Feature selection by L2-logisitic regression on training set
        embeded_lr_selector = SelectFromModel(logisticL2)
        embeded_lr_selector.fit(X_train, y_train)
        embeded_lr_support = embeded_lr_selector.get_support(indices=False)
        
        # Record selected features for stability measure
        features.append(embeded_lr_support)

        # Select only important features for the performance measure
        X_test = embeded_lr_selector.transform(X_test)
        X_train = embeded_lr_selector.transform(X_train)

        # Model fitting
        logisticL2.fit(X_train, y_train)
        y_pred = logisticL2.predict(X_test)
    
        # Record performance for iteration i
        performance.append(balanced_accuracy_score(y_test,y_pred))
        precision.append(precision_score(y_test,y_pred))
        recall.append(recall_score(y_test,y_pred))
        
        matrix.append(confusion_matrix(y_test, y_pred))
        
performance = np.asarray(performance)
precision = np.array(precision)
recall = np.array(recall)


In [ ]:
# Save selected feaures to calculate Phi
# http://www.cs.man.ac.uk/~gbrown/stability/
df = 1*pd.DataFrame(features,  columns=None)
df.to_csv("features_stability.csv")

In [ ]:
import stability as st
# help(st)
stab=st.getStability(np.array(df))
print('Stability of ENG_bin is :',stab)
varstab = st.getVarianceofStability(np.array(df))
print('Variance of the random procedure with M=10 is:',varstab['variance'])

In [ ]:
    
print ('BAcc Mean', performance.mean())
print ('BAcc Std', performance.std())
print ('BAcc Median', np.median(performance))

print ('Precision Mean', precision.mean())
print ('Precision Std', precision.std())
print ('Precision Median', np.median(precision))

print ('Recall Mean', recall.mean())
print ('Recall Std', recall.std())
print ('Recall Median', np.median(recall))

In [ ]:
CM = np.mean(matrix, axis=0)
ENG1 = CM[0,0]+CM[0,1]
ENG2 = CM[1,0]+CM[1,1]
CM[0,0] = CM[0,0]/ENG1*100
CM[0,1] = CM[0,1]/ENG1*100
CM[1,0] = CM[1,0]/ENG2*100
CM[1,1] = CM[1,1]/ENG2*100
CM

In [ ]:
# Distribution display
histo = performance
histo = np.round(histo,3)
histo = ["%.1f" % x for x in histo]

fig, ax = plt.subplots(1, 1, figsize=(15,15)) 
sns.set(style="whitegrid", color_codes=True, font_scale=2)


data = pd.DataFrame(histo, columns=['Performance'])
data = data.groupby("Performance").size()   # data underlying bar plot in question

pal = sns.color_palette("YlGn_r", len(data)) # Balanced Accuracy
rank = data.argsort().argsort()   # http://stackoverflow.com/a/6266510/1628638
b = sns.barplot(x=data.index, y=data, palette=np.array(pal[::-1])[rank])

b.axes.set_title("Feature Selection Workflow - Model balanced accuracy performance",fontsize=20)
b.set_xlabel("Balanced Accuracy of each 5-CV fold",fontsize=30)
b.set_ylabel("Iterations",fontsize=30)

In [ ]:
out = np.empty(shape=(9,2),dtype='object')
for i in range(9):
    out[i,0] = i
    out[i,1] = int(df[[i]].sum())

In [ ]:
auc_dataframe = pd.DataFrame(performance,  columns=None)
freq = pd.DataFrame(out,  columns=['Index','Frequency'])
freq['Scores']=columns
freq = freq.sort_values('Frequency', ascending=False)
# Let's plot the ranking of the features
sns.set(rc={'figure.figsize':(25,12)})
sns.set(style="whitegrid")
sns.barplot(y = freq.iloc[:,2], x= freq.iloc[:,1], palette='coolwarm_r')

In [ ]:
freq

# **Performance on Selected Features**

In [ ]:
X,y = X_dataset[["VMI","NAM","TMT","AMI"]],y

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
##########################################################################################
model = LogisticRegression(penalty="none", class_weight='balanced', n_jobs=-1)

# evaluate the model using k-fold cross-validation repeated 100 times and balanced accuracy

scores=[]

matrix=[]

for i in range (1,100):
    cv=StratifiedKFold(n_splits=5, random_state=i, shuffle=True)
    
    y_pred = cross_val_predict(model, X, y, cv=cv)
    matrix.append(confusion_matrix(y, y_pred))

# evaluate the model using k-fold cross-validation repeated 1000 times and roc AUC
  
scores=[]
for i in range (1,100):
    cv=StratifiedKFold(n_splits=5, random_state=i, shuffle=True)
    scores_temp=[]
    scores_temp=cross_val_score(model, X, y, scoring='balanced_accuracy',cv=cv)
    scores.append(scores_temp.mean())      

scores = np.asarray(scores)

print (f'Overall BACC mean :{scores.mean():.3f}')
print (f'Overall BACC std :{scores.std():.3f}')

In [ ]:
CM = np.mean(matrix, axis=0)
ENG1 = CM[0,0]+CM[0,1]
ENG2 = CM[1,0]+CM[1,1]
CM[0,0] = CM[0,0]/ENG1*100
CM[0,1] = CM[0,1]/ENG1*100
CM[1,0] = CM[1,0]/ENG2*100
CM[1,1] = CM[1,1]/ENG2*100
CM

# **Partial Dependence Plots on complete Dataset with 5-CV subsampling**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=0)

In [ ]:
model = LogisticRegression(penalty="l2", solver='liblinear', class_weight='balanced', n_jobs=-1)
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
balanced_accuracy_score(y_test, y_predicted)

In [ ]:
plt.rcParams['figure.figsize'] = (25,25)

print('Computing partial dependence plots...')

features = ["AMI","VMI","TMT","NAM",('AMI','VMI'),('AMI','TMT'),('AMI','NAM'),('VMI','TMT'),('VMI','NAM'),('TMT','NAM')]
plot_partial_dependence(model, X_train, features,n_jobs=-1, grid_resolution=20)
fig = plt.gcf()
fig.suptitle('Partial dependence plots')
fig.subplots_adjust(hspace=0.3)

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
fig = plt.figure()

features = ('VMI', 'AMI')
pdp, axes = partial_dependence(model, X_train, features=features,
                               grid_resolution=20)
XX, YY = np.meshgrid(axes[0], axes[1])
Z = pdp[0].T
ax = Axes3D(fig)
surf = ax.plot_surface(XX, YY, Z, rstride=1, cstride=1,
                       cmap=plt.cm.BuPu, edgecolor='k')
ax.set_xlabel(features[0])
ax.set_ylabel(features[1])
ax.set_zlabel('Partial dependence')
#  pretty init view
ax.view_init(elev=22, azim=122)
plt.colorbar(surf)
plt.suptitle('Partial dependence')
plt.subplots_adjust(top=0.9)

plt.show()